Notebook to collect the information of the generation

### Import

In [1]:
import sys, os, re
from glob import glob
sys.path.append('../lib')

import time, datetime

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import humanfriendly

In [3]:
import uproot as ur
import ROOT as rt
rt.gErrorIgnoreLevel = rt.kError
rt.RooMsgService.instance().setGlobalKillBelow(rt.RooFit.ERROR)
import root_numpy as rtnp

Welcome to JupyROOT 6.10/08


# Inputs

In [4]:
class Bauble: pass

In [5]:
# data_dir = '../data/cmsMC_private/'
# generation_tag = 'BPH_Tag-B0_*pythia8_Hardbbbar_PTFilter5_0p0-evtgen_ISGW2_PU35_10-2-3'
# aux = glob(data_dir+generation_tag)
# print aux
# if 'TauNu' in aux[1]:
#     muDir = aux[0]
#     tauDir = aux [1]
# else:
#     muDir = aux[1]
#     tauDir = aux [0]
    
# outDic = {}
# outDic['mu'] = Bauble()
# outDic['mu'].dir = muDir
# outDic['tau'] = Bauble()
# outDic['tau'].dir = tauDir

In [6]:
# outDic = {}
# outDic['B0'] = Bauble()
# outDic['B0'].dir = '../data/cmsMC_private/BPH_Tag-Mu_Probe-B0_KDmst-pD0bar-kp_13TeV-pythia8_Hardbbbar_PTFilter5_0p0-evtgen_SVS_PU0_10-2-3_v0'

In [7]:
outDic = {}
outDic['B02JpsiKst'] = Bauble()
outDic['B02JpsiKst'].dir = '../data/cmsMC_private/'+'BPH_Tag-Probe_B0_JpsiKst-mumuKpi-kp_13TeV-pythia8_SoftQCD_PTFilter5_0p0-evtgen_SVV_PU20_10-2-3'
# 'BPH_Tag-Probe_B0_JpsiKst-mumuKpi-kp_13TeV-pythia8_Hardbbbar_PTFilter5_0p0-evtgen_SVV_PU20_10-2-3_v0'

# Efficiency

In [8]:
def getEff(k,N):
    e = k/float(N)
    de = np.sqrt(e*(1-e)/N)
    return [e, de]

In [9]:
for name in outDic.keys():
    print '\n--> ' + name
    indir = outDic[name].dir + '/'

    step1_log_lines = open(indir + 'step1.log').readlines()

    for line in reversed(step1_log_lines):
        if 'Pythia::next():' in line:
            break
    outDic[name].N_gen = int(line[17:-29])
    print outDic[name].N_gen

    for line in reversed(step1_log_lines):
        if 'Begin processing the ' in line:
            break
    aux = re.search(r', Event [0-9]+,', line).group(0)
    outDic[name].N_gen_cuts = int(aux[8:-1]) + 99
    print outDic[name].N_gen_cuts

    f = rt.TFile(indir + 'test_MINIAODSIM.root')
    t = f.Get('Events')
    outDic[name].N_acc = int(t.GetEntries())
    print outDic[name].N_acc

    step5_log_lines = open(indir + 'step5.log').readlines()
    eff_ln = []
    for line in reversed(step5_log_lines):
        if 'efficiency:' in line:
            eff_ln.append(line)

    aux = re.search(': [0-9]+/', eff_ln[1]).group(0)
    outDic[name].N_trg = int(aux[2:-1])
    print outDic[name].N_trg

    aux = re.search(': [0-9]+/', eff_ln[0]).group(0)
    outDic[name].N_cand = int(aux[2:-1])
    print outDic[name].N_cand


--> B02JpsiKst
259000
20000
311
99
23


# Time

In [10]:
def bashDate2Unix(ln):
    ln = ln.replace('CEST ', '')
    t = time.mktime(datetime.datetime.strptime(ln, template).timetuple())
    return float(t)

In [11]:
for d in outDic.values():
    indir = d.dir + '/'
    test_log_lines = open(indir + 'test.log').readlines()

    d.crono = [[],[],[],[],[]]

    template = '%c'
    for i, ln in enumerate(test_log_lines):
        for j in range(1,5):
            if 'Step '+str(j) in ln or 'Running step '+str(j) in ln:
#                 print ln[:-1]
#                 print test_log_lines[i+1][:-1]
                d.crono[j-1].append(bashDate2Unix(test_log_lines[i+1][:-1]))
    d.crono[4].append(bashDate2Unix(test_log_lines[-1][:-1]))
    
    d.driver_time = []
    d.run_time = []
    for i in range(4):
        d.driver_time.append(d.crono[i][1] - d.crono[i][0])
        d.run_time.append(d.crono[i+1][0] - d.crono[i][1])

# Print

In [12]:
for n, d in outDic.iteritems():
    table = PrettyTable()
    table.field_names = ['Step', 'Sub', 'Eff [%]', 'Driver time [s]', 'Running time/evt [ms]']
    e, de = getEff(d.N_acc, d.N_gen_cuts)
    table.add_row(['GEN-SIM', '-', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '{:.1f}'.format(d.driver_time[0]), '{:.1f}'.format(1e3*d.run_time[0]/d.N_gen_cuts)])
    e, de = getEff(d.N_gen_cuts, d.N_gen)
    table.add_row(['', 'Pythia Gen', 'x {:.1f}'.format(1./e), '-', '-'])
    table.add_row(['', 'Gen Filter', '100.0', '-', '-'])
    e, de = getEff(d.N_acc, d.N_gen_cuts)
    table.add_row(['', 'CMSSW Filter', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '-', '-'])

    table.add_row(['RAW', '-', '100.0', '{:.1f}'.format(d.driver_time[1]), '{:.1f}'.format(1e3*d.run_time[1]/d.N_acc)])
    table.add_row(['AOD', '-', '100.0', '{:.1f}'.format(d.driver_time[2]), '{:.1f}'.format(1e3*d.run_time[2]/d.N_acc)])
    table.add_row(['MINIAOD', '-', '100.0', '{:.1f}'.format(d.driver_time[3]), '{:.1f}'.format(1e3*d.run_time[3]/d.N_acc)])
    
    e, de = getEff(d.N_cand, d.N_acc)
    table.add_row(['CAND', '-', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '-', '-'])
    e, de = getEff(d.N_trg, d.N_acc)
    table.add_row(['', 'BPH Trg', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '-', '-'])
    e, de = getEff(d.N_cand, d.N_trg)
    table.add_row(['', 'Cand. sel.', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '-', '-'])
    
    table.add_row(len(table.field_names)*[''])
    
    e, de = getEff(d.N_cand, d.N_gen_cuts)
    table.add_row(['Tot', '', '{:.3f} +/- {:.3f}'.format(100*e, 100*de), '{:.1f}'.format(np.sum(d.driver_time)), '{:.1f}(*)'.format(1e3*np.sum(d.run_time)/d.N_cand)])

    print n
    print table.get_string()
    
    tabsum = PrettyTable()
    tabsum.field_names = ['Evts req.', '# MINIAOD Evts', '# Cand.', 'Tot. Time [min]']
    
    num_bytes = os.path.getsize(d.dir + '/test_MINIAODSIM.root')
    MINIAOD_size = humanfriendly.format_size(num_bytes, binary=True)
    num_bytes = os.path.getsize(glob(d.dir + '/test_*CAND.root')[0])
    B02DstMuCAND_size = humanfriendly.format_size(num_bytes, binary=True)
    
    tabsum.add_row(['{:.0f}k'.format(1e-3*d.N_gen_cuts), '{:.0f} ({})'.format(d.N_acc, MINIAOD_size), '{:.0f} ({})'.format(d.N_cand, B02DstMuCAND_size), '{:.0f}'.format((d.crono[-1][-1]-d.crono[0][0])/60.)])
    print tabsum.get_string()
    
    
    print

B02JpsiKst
+---------+--------------+-----------------+-----------------+-----------------------+
|   Step  |     Sub      |     Eff [%]     | Driver time [s] | Running time/evt [ms] |
+---------+--------------+-----------------+-----------------+-----------------------+
| GEN-SIM |      -       |  1.55 +/- 0.09  |       51.0      |         379.7         |
|         |  Pythia Gen  |      x 13.0     |        -        |           -           |
|         |  Gen Filter  |      100.0      |        -        |           -           |
|         | CMSSW Filter |  1.55 +/- 0.09  |        -        |           -           |
|   RAW   |      -       |      100.0      |       24.0      |         1900.3        |
|   AOD   |      -       |      100.0      |       40.0      |         858.5         |
| MINIAOD |      -       |      100.0      |       16.0      |         755.6         |
|   CAND  |      -       |  7.40 +/- 1.48  |        -        |           -           |
|         |   BPH Trg    |  31.8